In [1]:
import os
import numpy as np
caffe_root = '/home/flathead/bitbucket/mycaffe/'
os.chdir('../../')
import sys
sys.path.insert(0, 'python')
import caffe

In [2]:
from pylab import *
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
from numpy.random import normal

In [4]:
caffe.set_mode_gpu()
caffe.set_device(0)

In [ ]:
solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
solver = caffe.SGDSolver('examples/mlp/mlp_solver.prototxt')
net1 = solver.net
test_net = solver.test_nets[0]

In [5]:
net1 = caffe.Net('examples/mlp/mlp_train_test.prototxt',
                'examples/mlp/baseline.caffemodel', 
                 caffe.TEST)

In [ ]:
# during training, each output is (batch size, feature dim, spatial dim)
[(k, v.data.shape) for k, v in net1.blobs.items()]

In [ ]:
# just print the weight sizes (we'll omit the biases)
[(k, v[0].data.shape) for k, v in net1.params.items()]

In [ ]:
net1.forward()

In [6]:
feat1 = net1.params['ip1'][0].data
feat2 = net1.params['ip2'][0].data
feat3 = net1.params['ip3'][0].data
feat4 = net1.params['ip4'][0].data

In [7]:
l1 = 500
l2 = 500
l3 = 2000
l4 = 10

In [ ]:
plt.figure(figsize=(15, 15))
#plt.hist(train_net.params['ip1'][0].data)
'''
for j in range(1,21):
    plt.subplot(21,1,j)
    plt.hist(feat1[j-1], bins = 784)
    plt.title(j)
'''
plt.subplot(4,1,1)
plt.hist(feat1.flat, bins = 500)
plt.title("Input layer")

plt.subplot(4,1,2)
plt.hist(feat2.flat, bins = 500)
plt.title("1'st layer")

plt.subplot(4,1,3)
plt.hist(feat3.flat, bins = 500)
plt.title("2'nd layer")

plt.subplot(4,1,4)
plt.hist(feat4.flat, bins = 500)
plt.title("Last layer")

In [ ]:
%%time
niter = 5000
test_interval = 500
train_loss = zeros(niter)
test_acc = zeros(int(np.ceil(niter / test_interval)))
output = zeros((niter, 8, 10))

for it in range(niter):
    solver.step(1)
    net1.forward()
    
    train_loss[it] = net1.blobs['loss'].data
    
    net1.forward(start='ip1')
    output[it] = test_net.blobs['ip4'].data[:8]
    
    if it % test_interval == 0:
        print 'iteration', it, 'testing...'
        correct = 0
        for test_it in range(100):
            test_net.forward()
            correct += sum(test_net.blobs['ip4'].data.argmax(1) == test_net.blobs['label'].data)
        test_acc[it // test_interval] = correct / 1e4

In [ ]:
net1.save('examples/mlp/meanpr/baseline.caffemodel')

In [8]:
#Neurons of Layer 1
mean_feat1 = [0] * l1
std_feat1 = [0] * l1

for j in range(l1):
    mean_feat1[j] = feat1[j].mean()
    std_feat1[j] = feat1[j].std()
    
#Neurons of Layer 2
mean_feat2 = [0] * l2
std_feat2 = [0] * l2

for j in range(l2):
    mean_feat2[j] = feat2[j].mean()
    std_feat2[j] = feat2[j].std()
    
#Neurons of Layer 3
mean_feat3 = [0] * l3
std_feat3 = [0] * l3

for j in range(l3):
    mean_feat3[j] = feat3[j].mean()
    std_feat3[j] = feat3[j].std()
    
#Neurons of Layer 4
mean_feat4 = [0] * l4
std_feat4 = [0] * l4

for j in range(l4):
    mean_feat4[j] = feat4[j].mean()
    std_feat4[j] = feat4[j].std()

In [ ]:
#Normalize
for j in range(l1):
    feat1[j] /= 28
    
for j in range(l2):
    feat2[j] /= 4.47213

for j in range(l3):
    feat3[j] /= 4.47213

In [ ]:
#Normalized Variance
for j in range(l1):
    print feat1[j].var()

In [9]:
#Neurons of Layer 1
posm_feat1 = [0] * l1
negm_feat1 = [0] * l1
poss_feat1 = [0] * l1
negs_feat1 = [0] * l1

for j in range(l1):
    posm_feat1[j] = feat1[j][feat1[j] > mean_feat1[j]].mean()
    negm_feat1[j] = feat1[j][feat1[j] < mean_feat1[j]].mean()
    poss_feat1[j] = feat1[j][feat1[j] > mean_feat1[j]].std()
    negs_feat1[j] = feat1[j][feat1[j] < mean_feat1[j]].std()
    
#Neurons of Layer 2
posm_feat2 = [0] * l2
negm_feat2 = [0] * l2
poss_feat2 = [0] * l2
negs_feat2 = [0] * l2

for j in range(l2):
    posm_feat2[j] = feat2[j][feat2[j] > mean_feat2[j]].mean()
    negm_feat2[j] = feat2[j][feat2[j] < mean_feat2[j]].mean()
    poss_feat2[j] = feat2[j][feat2[j] > mean_feat2[j]].std()
    negs_feat2[j] = feat2[j][feat2[j] < mean_feat2[j]].std()
    
#Neurons of Layer 3
posm_feat3 = [0] * l3
negm_feat3 = [0] * l3
poss_feat3 = [0] * l3
negs_feat3 = [0] * l3

for j in range(l3):
    posm_feat3[j] = feat3[j][feat3[j] > mean_feat3[j]].mean()
    negm_feat3[j] = feat3[j][feat3[j] < mean_feat3[j]].mean()
    poss_feat3[j] = feat3[j][feat3[j] > mean_feat3[j]].std()
    negs_feat3[j] = feat3[j][feat3[j] < mean_feat3[j]].std()

#Neurons of Layer 4
posm_feat4 = [0] * l4
negm_feat4 = [0] * l4
poss_feat4 = [0] * l4
negs_feat4 = [0] * l4

for j in range(l4):
    posm_feat4[j] = feat4[j][feat4[j] > mean_feat4[j]].mean()
    negm_feat4[j] = feat4[j][feat4[j] < mean_feat4[j]].mean()
    poss_feat4[j] = feat4[j][feat4[j] > mean_feat4[j]].std()
    negs_feat4[j] = feat4[j][feat4[j] < mean_feat4[j]].std()

In [ ]:
#평행이동
for x in range(l1):
    feat1[x] = (feat1[x] - mean_feat1[x])
    #feat1[x] = (feat1[x] - mean_feat1[x]) / std_feat1[x]
for y in range(l2):
    feat2[y] = (feat2[y] - mean_feat2[y])
    #feat2[y] = (feat2[y] - mean_feat2[y]) / std_feat2[y]

In [ ]:
#평행이동
net1.save('examples/mlp/meanpr/shift.caffemodel')

In [ ]:
#Neurons of Layer 1
mean_feat1 = [0] * 20
std_feat1 = [0] * 20

for j in range(20):
    mean_feat1[j] = feat1[j].mean()
    std_feat1[j] = feat1[j].std()
    
#Neurons of Layer 2
mean_feat2 = [0] * 50
std_feat2 = [0] * 50

for j in range(50):
    mean_feat2[j] = feat2[j].mean()
    std_feat2[j] = feat2[j].std()

In [ ]:
for j in range(20):
#    print [mean_feat1[j] - 0.5 * std_feat1[j], mean_feat1[j] + 0.5 * std_feat1[j]]
    print [negm_feat1[j]-negs_feat1[j], negm_feat1[j]+negs_feat1[j], mean_feat1[j]-std_feat1[j], mean_feat1[j]+std_feat1[j], posm_feat1[j]-poss_feat1[j], posm_feat1[j]+poss_feat1[j]]

In [ ]:
for j in range(50):
    print [negm_feat2[j]-negs_feat2[j], negm_feat2[j]+negs_feat2[j], mean_feat2[j]-std_feat2[j], mean_feat2[j]+std_feat2[j], posm_feat2[j]-poss_feat2[j], posm_feat2[j]+poss_feat2[j]]

In [10]:
# 0.5 * std range mean pruning
r = 0.1
'''
a1 = [0] * l1
b1 = [0] * l2
c1 = [0] * l3
d1 = [0] * l4
a2 = [0] * l1
b2 = [0] * l2
c2 = [0] * l3
d2 = [0] * l4

for x in range(l1):
    a1[x] = (posm_feat1[x] - mean_feat1[x]) / 2 / std_feat1[x]
    a2[x] = (mean_feat1[x] - negm_feat1[x]) / 2 / std_feat1[x]
for y in range(l2):
    b1[y] = (posm_feat2[y] - mean_feat2[y]) / 2 / std_feat2[y]
    b2[y] = (mean_feat2[y] - negm_feat2[y]) / 2 / std_feat2[y]
for z in range(l3):
    c1[z] = (posm_feat3[z] - mean_feat3[z]) / 2 / std_feat3[z]
    c2[z] = (mean_feat3[z] - negm_feat3[z]) / 2 / std_feat3[z]
for z in range(l4):
    d1[z] = (posm_feat4[z] - mean_feat4[z]) / 2 / std_feat4[z]
    d2[z] = (mean_feat4[z] - negm_feat4[z]) / 2 / std_feat4[z]
'''    
for x in range(l1):
    for y in range(784):
        #if (feat1[x,y] >= (mean_feat1[x] - a2[x] * std_feat1[x])) & (feat1[x,y] <= (mean_feat1[x] + a1[x] * std_feat1[x])):
        if (feat1[x,y] >= -r * std_feat1[x]) & (feat1[x,y] <= r * std_feat1[x]):
            #feat1[x,y] = mean_feat1[x]
            feat1[x,y] = 0
            
for x in range(l2):
    for y in range(l1):
        #if (feat2[x,y] >= (mean_feat2[x] - b2[x] * std_feat2[x])) & (feat2[x,y] <= (mean_feat2[x] + b1[x] * std_feat2[x])):
        if (feat2[x,y] >= -r * std_feat2[x]) & (feat2[x,y] <= r * std_feat2[x]):
            #feat2[x,y] = mean_feat2[x]
            feat2[x,y] = 0
    
for x in range(l3):
    for y in range(l2):
        #if (feat3[x,y] >= (mean_feat3[x] - c2[x] * std_feat3[x])) & (feat3[x,y] <= (mean_feat3[x] + c1[x] * std_feat3[x])):
        if (feat3[x,y] >= -r * std_feat3[x]) & (feat3[x,y] <= r * std_feat3[x]):
            #feat3[x,y] = mean_feat3[x]
            feat3[x,y] = 0
            
for x in range(l4):
    for y in range(l3):
        #if (feat4[x,y] >= (mean_feat4[x] - d2[x] * std_feat4[x])) & (feat4[x,y] <= (mean_feat4[x] + d1[x] * std_feat4[x])):
        if (feat4[x,y] >= -r * std_feat4[x]) & (feat4[x,y] <= r * std_feat4[x]):
            #feat4[x,y] = mean_feat4[x]
            feat4[x,y] = 0

In [11]:
net1.save('examples/mlp/meanpr/prune.caffemodel')

In [ ]:
%%time
niter = 10000
test_interval = 500
train_loss = zeros(niter)
test_acc = zeros(int(np.ceil(niter / test_interval)))
output = zeros((niter, 8, 10))

for it in range(niter):
    solver.step(1)
    
    train_loss[it] = solver.net.blobs['loss'].data
    
    solver.test_nets[0].forward(start='conv1')
    output[it] = solver.test_nets[0].blobs['ip3'].data[:8]
    
    if it % test_interval == 0:
        print 'iteration', it, 'testing...'
        correct = 0
        for test_it in range(100):
            solver.test_nets[0].forward()
            correct += sum(solver.test_nets[0].blobs['ip3'].data.argmax(1) == solver.test_nets[0].blobs['label'].data)
        test_acc[it // test_interval] = correct / 1e4

In [ ]:
plt.figure(figsize=(15, 15))
#plt.hist(train_net.params['ip1'][0].data)
'''
for j in range(1,21):
    plt.subplot(21,1,j)
    plt.hist(feat1[j-1], bins = 784)
    plt.title(j)
'''
plt.subplot(4,1,1)
plt.hist(feat1.flat, bins = 500)
plt.title("Input layer")

plt.subplot(4,1,2)
plt.hist(feat2.flat, bins = 500)
plt.title("1'st layer")

plt.subplot(4,1,3)
plt.hist(feat3.flat, bins = 500)
plt.title("2'nd layer")

plt.subplot(4,1,4)
plt.hist(feat4.flat, bins = 500)
plt.title("Last layer")

In [ ]:
for x in range(l1):
    print (feat1[x] == mean_feat1[x]).sum()
for y in range(l2):
    print (feat2[y] == mean_feat2[y]).sum()
for z in range(l3):
    print (feat3[y] == mean_feat3[y]).sum()

In [ ]:
#Neurons of Layer 1
posm_feat1 = [0] * l1
negm_feat1 = [0] * l1
poss_feat1 = [0] * l1
negs_feat1 = [0] * l1

for j in range(l1):
    posm_feat1[j] = feat1[j][feat1[j] > mean_feat1[j]].mean()
    negm_feat1[j] = feat1[j][feat1[j] < mean_feat1[j]].mean()
    poss_feat1[j] = feat1[j][feat1[j] > mean_feat1[j]].std()
    negs_feat1[j] = feat1[j][feat1[j] < mean_feat1[j]].std()
    
#Neurons of Layer 2
posm_feat2 = [0] * l2
negm_feat2 = [0] * l2
poss_feat2 = [0] * l2
negs_feat2 = [0] * l2

for j in range(l2):
    posm_feat2[j] = feat2[j][feat2[j] > mean_feat2[j]].mean()
    negm_feat2[j] = feat2[j][feat2[j] < mean_feat2[j]].mean()
    poss_feat2[j] = feat2[j][feat2[j] > mean_feat2[j]].std()
    negs_feat2[j] = feat2[j][feat2[j] < mean_feat2[j]].std()
    
#Neurons of Layer 3
posm_feat3 = [0] * l3
negm_feat3 = [0] * l3
poss_feat3 = [0] * l3
negs_feat3 = [0] * l3

for j in range(l3):
    posm_feat3[j] = feat3[j][feat3[j] > mean_feat3[j]].mean()
    negm_feat3[j] = feat3[j][feat3[j] < mean_feat3[j]].mean()
    poss_feat3[j] = feat3[j][feat3[j] > mean_feat3[j]].std()
    negs_feat3[j] = feat3[j][feat3[j] < mean_feat3[j]].std()
    
#Neurons of Layer 4
posm_feat4 = [0] * l4
negm_feat4 = [0] * l4
poss_feat4 = [0] * l4
negs_feat4 = [0] * l4

for j in range(l4):
    posm_feat4[j] = feat4[j][feat4[j] > mean_feat4[j]].mean()
    negm_feat4[j] = feat4[j][feat4[j] < mean_feat4[j]].mean()
    poss_feat4[j] = feat4[j][feat4[j] > mean_feat4[j]].std()
    negs_feat4[j] = feat4[j][feat4[j] < mean_feat4[j]].std()

In [ ]:
for x in range(l1):
    for y in range(784):
        if (feat1[x, y] > 0):
            feat1[x,y] = posm_feat1[x]
        if (feat1[x, y] < 0):
            feat1[x,y] = negm_feat1[x]
for x in range(l2):
    for y in range(l1):
        if (feat2[x, y] > 0):
            feat2[x,y] = posm_feat2[x]
        if (feat2[x, y] < 0):
            feat2[x,y] = negm_feat2[x]
for x in range(l3):
    for y in range(l2):
        if (feat3[x, y] > 0):
            feat3[x,y] = posm_feat3[x]
        if (feat3[x, y] < 0):
            feat3[x,y] = negm_feat3[x]
for x in range(l4):
    for y in range(l3):
        if (feat4[x, y] > 0):
            feat4[x,y] = posm_feat4[x]
        if (feat4[x, y] < 0):
            feat4[x,y] = negm_feat4[x]

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(4,1,1)
plt.hist(feat1[0].flat, bins = 500)
plt.title("Input layer")

plt.subplot(4,1,2)
plt.hist(feat2[0].flat, bins = 500)
plt.title("1'st layer")

plt.subplot(4,1,3)
plt.hist(feat3[0].flat, bins = 500)
plt.title("2'nd layer")

plt.subplot(4,1,4)
plt.hist(feat4[0].flat, bins = 500)
plt.title("Last layer")

In [ ]:
net1.save('examples/mlp/meanpr/quant.caffemodel')

In [ ]:
for x in range(l1):
    print ((feat1[x] == posm_feat1[x]) | (feat1[x] == negm_feat1[x]) | (feat1[x] == mean_feat1[x])).sum()
for y in range(l2):
    print ((feat2[y] == posm_feat2[y]) | (feat2[y] == negm_feat2[y]) | (feat2[y] == mean_feat2[y])).sum()
for z in range(l3):
    print ((feat3[z] == posm_feat3[z]) | (feat3[z] == negm_feat3[z]) | (feat3[z] == mean_feat3[z])).sum()

In [ ]:
for j in range(20):
    #print feat2[j].mean()
    #print feat2[j].std()
    print feat1[j][feat1[j] > std_feat1[j]].mean()
    print feat1[j][feat1[j] < -std_feat1[j]].mean()